In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as st
import plotly.offline as py
import plotly.graph_objs as go
import os
import re

In [ ]:
py.init_notebook_mode(connected=True)

In [ ]:
DATA_PATH = os.path.join('data','wr201126')
DATA_MAIN_PATH = os.path.join(DATA_PATH, 'wr201126.txt')
DATA_METEOSTATIONS_PATH = os.path.join(DATA_PATH, 'statlist201126.txt')
DATA_FIELDS_PATH = os.path.join(DATA_PATH, 'fld201126a0.txt')

In [ ]:
assert os.path.isdir(DATA_PATH)
assert os.path.isfile(DATA_MAIN_PATH)
assert os.path.isfile(DATA_METEOSTATIONS_PATH)
assert os.path.isfile(DATA_FIELDS_PATH)

In [ ]:
meteostations = []
with open(DATA_METEOSTATIONS_PATH, encoding="WINDOWS-1251") as f:
    for line in f.readlines():
        m_data = re.split("\s+", line)[:-1]
        if len(m_data) > 3:
            m_name = " ".join(m_data[1:-2]).strip()
            meteostations.append([m_data[0], m_name, m_data[-1]])
        else:
            meteostations.append(m_data)
for m_data in meteostations:
    assert len(m_data) == 3
df_meteostations = pd.DataFrame(meteostations, columns=["Индекс ВМО", "Название станции", "Страна"])

In [ ]:
header = []
with open(DATA_FIELDS_PATH, encoding="WINDOWS-1251") as f:
    header = [ " ".join(re.split("\s+", s)[4:]).strip() for s in f.readlines() ]
assert len(header) == 9

In [ ]:
df = pd.read_csv(DATA_MAIN_PATH, sep=";", header=None, names=header)

In [ ]:
df

In [ ]:
df_meteostations[df_meteostations["Название станции"].str.contains("Москва")]

In [ ]:
date_column = pd.DataFrame(pd.to_datetime(df.Год*10000+df.Месяц*100+df.День,format='%Y%m%d'))
df.insert(1,"Дата",date_column)
df = df.set_index("Дата")

In [ ]:
df.dropna()
df = df.drop(["Год","Месяц", "День"], axis=1)

In [ ]:
df

In [ ]:
df = df[df["Общий признак качества температур"] != 9]
df = df.drop(["Общий признак качества температур"], axis=1)

In [ ]:
_tp_cols = header[5:9]
_t_cols = _tp_cols[0:3]
_p_col = _tp_cols[3]

print(_tp_cols, _t_cols, _p_col)

In [ ]:
print("Количество пропусков в данных")
print("-----------------------------")
for _col in _tp_cols:
    n_voids = 0
    for value in df[_col].tolist():
        if re.match("^\s+$", value):
            n_voids += 1
    print("{}: {} из {} ({:.2}%)".format(_col, n_voids, df.shape[0], n_voids * 100.0 / float(df.shape[0])))

In [ ]:
v = "     "
dft = df[(df[_t_cols[0]] != v) & (df[_t_cols[1]] != v) & (df[_t_cols[2]] != v)]
dft = dft.drop([_p_col], axis=1)
dfp = df[ df[_p_col] != v ]
dfp = dfp.drop(_t_cols, axis=1)
dftp = df[(df[_t_cols[0]] != v) & (df[_t_cols[1]] != v) & (df[_t_cols[2]] != v) & (df[_p_col] != v) ]

In [ ]:
dftp.info()

In [ ]:
for _column in _t_cols:
    dft[_column] = dft[_column].astype(float)
dfp[_p_col] = dfp[_p_col].astype(float)
for _column in _tp_cols:
    dftp[_column] = dftp[_column].astype(float)
dftp[_p_col] = dftp[_p_col].astype(float)

In [ ]:
data = dft[_t_cols].resample("Y").mean()
layout = go.Layout(
    title="График изменения температуры воздуха (по годам)",
    xaxis=dict(title = "Дата", type='date'),
    yaxis=dict(title = "Температура воздуха, °C"),
)
temperature_traces = []
for _t_col in _t_cols:
    temperature_traces.append(go.Scatter(x = data.index, y = data[_t_col], name = _t_col))
py.iplot(go.Figure(data=temperature_traces, layout=layout))

In [ ]:
data = dft[[_t_cols[1]]].resample("Y").mean().rolling(10, min_periods=10).mean().dropna()

layout = go.Layout(
    title="График изменения среднегодовой температуры воздуха усредненной за период декады (10 лет)",
    xaxis=dict(title = "Время, год", type='date'),
    yaxis=dict(title = "Температура воздуха, °C"),
)

trace1 = go.Scatter(x = data.index, y=data[_t_cols[1]], name=_t_cols[1])
py.iplot(go.Figure(data=[trace1], layout=layout))

In [ ]:
data = dft[dft.index > "1988-01-01"][[_t_cols[1]]].resample("Y").mean().rolling(10, min_periods=10).mean().dropna()

temperature = data[_t_cols[1]].tolist()
idx = np.arange(0,len(temperature))
slope, intercept, r_value, p_value, std_err = st.linregress(idx,temperature)
line = slope * idx + intercept

layout = go.Layout(
    title="График изменения среднегодовой температуры воздуха усредненной за период декады (10 лет)",
    xaxis=dict(title = "Время, год", type='date'),
    yaxis=dict(title = "Температура воздуха, °C"),
)

trace1 = go.Scatter(x = data.index, y=data[_t_cols[1]], name=_t_cols[1])
trace2 = go.Scatter(x = data.index, y=line, name='Линия тренда')
py.iplot(go.Figure(data=[trace1, trace2], layout=layout))

In [ ]:
print("Температура воздуха c 1988-го года возрастает в среднем на {} °C за год".format(round(slope,2)))

In [ ]:
print("Через 10 лет средняя среднегодовая приповерхностная температура воздуха составит {:.2}°C".format(data[-1:][_t_cols[1]][0] + round(slope,2) * 77))

In [ ]:
data = dfp[[_p_col]].resample("Y").sum() / 600
layout = go.Layout(
    title="График изменения суммы осадков (по годам)",
    xaxis=dict(title = "Дата", type='date'),
    yaxis=dict(title = "Атмосферные осадки, мм"),
)
precipitation_traces = []
precipitation_traces.append(go.Scatter(x = data.index, y = data[_p_col], name = _p_col))
py.iplot(go.Figure(data=precipitation_traces, layout=layout))

In [ ]:
data = dfp[[_p_col]].resample("Y").sum().rolling(10, min_periods=10).mean().dropna() / 600

layout = go.Layout(
    title="График изменения годовой суммы атмосферных осадков усредненной за период декады (10 лет)",
    xaxis=dict(title = "Время, год", type='date'),
    yaxis=dict(title = "Атмосферные осадки, мм"),
)

trace1 = go.Scatter(x = data.index, y=data[_p_col], name=_p_col)
py.iplot(go.Figure(data=[trace1], layout=layout))

In [ ]:
data = dfp[dfp.index < "1975-01-01"][[_p_col]].resample("Y").sum().rolling(10, min_periods=10).mean().dropna() / 600

precipitation = data[_p_col].tolist()
idx = np.arange(0,len(precipitation))
slope, intercept, r_value, p_value, std_err = st.linregress(idx,precipitation)
line = slope * idx + intercept

layout = go.Layout(
    title="График изменения годовой суммы атмосферных осадков усредненной за период декады до 1975-го года",
    xaxis=dict(title = "Время, год", type='date'),
    yaxis=dict(title = "Атмосферные осадки, мм"),
)

trace1 = go.Scatter(x = data.index, y=data[_p_col], name="Исходные данные")
trace2 = go.Scatter(x = data.index, y=line, name='Линия тренда')
py.iplot(go.Figure(data=[trace1,trace2], layout=layout))

In [ ]:
print("Атмосферные осадки до 1975-го года возрастают в среднем на {} мм за год".format(round(slope,2)))

In [ ]:
print("За период с 1950 по 1975 годы атмосферные осадки выросли на {}%".format(int(100.0 * (475.0 - 349.0) / 349.0)))

In [ ]:
data = dfp[dfp.index > "1975-01-01"][[_p_col]].resample("Y").sum().rolling(10, min_periods=10).mean().dropna() / 600

precipitation = data[_p_col].tolist()
idx = np.arange(0,len(precipitation))
slope, intercept, r_value, p_value, std_err = st.linregress(idx,precipitation)
line = slope * idx + intercept

layout = go.Layout(
    title="График изменения годовой суммы атмосферных осадков усредненной за период декады после 1975-го года",
    xaxis=dict(title = "Время, год", type='date'),
    yaxis=dict(title = "Атмосферные осадки, мм"),
)

trace1 = go.Scatter(x = data.index, y=data[_p_col], name="Исходные данные")
trace2 = go.Scatter(x = data.index, y=line, name='Линия тренда')
py.iplot(go.Figure(data=[trace1,trace2], layout=layout))

In [ ]:
print("Атмосферные осадки с 1975-го года убывают в среднем на {} мм за год".format(round(-slope,2)))